<H1>1. Klasifikace zvukových záznamů  </H1>

Cílem této práce je klasifikace hlasových záznamů samohlásky „a“ z databáze VOICED
na zdravé a patologické pomocí příznaků v časové a frekvenční oblasti.
V případě patologických záznamů je dále provedena vícetřídní klasifikace jednotlivých poruch.
Zpracování signálu zahrnuje Fourierovu a kepstrální analýzu.
Výsledky klasifikace jsou porovnány s anotacemi databáze.


Import knihoven

In [5]:
import os
import numpy as np
import wfdb
import matplotlib.pyplot as plt
import pandas as pd
from scipy.fft import fft, ifft
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


Normalizace diagnóz

In [6]:
# Vylepšená funkce pro normalizaci (přidejte více klíčových slov z VOICED)
def normalize_diagnosis(diag):
    diag = diag.lower().strip()
    if any(x in diag for x in ["healthy", "normal", "n"]):
        return "healthy"
    if any(x in diag for x in ["dysphonia", "hyperfunctional"]):
        return "dysphonia"
    if any(x in diag for x in ["cord", "vocal", "paralysis", "reflux"]):
        return "vocal_cord_pathology"
    return "other_pathology"

# Při načítání v cyklu si vypište diagnózy pro kontrolu
# for comment in record.comments:
#     print(f"DEBUG: {comment}")

Rámování signálu, Časové příznaky a Spektrální + kepstrální příznaky

In [7]:
def frame_signal(signal, frame_size, hop_size):
    frames = []
    for start in range(0, len(signal) - frame_size, hop_size):
        frames.append(signal[start:start + frame_size])
    return np.array(frames)

def time_domain_features(signal):
    signal = signal - np.mean(signal)

    rms = np.sqrt(np.mean(signal**2))
    zcr = np.mean(np.abs(np.diff(np.sign(signal)))) / 2
    variance = np.var(signal)

    return rms, zcr, variance

def spectral_features(signal, fs):
    frame_size = int(0.03 * fs)    # 30 ms
    hop_size = int(0.015 * fs)     # 50 % overlap

    frames = frame_signal(signal, frame_size, hop_size)
    window = np.hamming(frame_size)
    centroids = []
    flatnesses = []
    cepstra = []

    freqs = np.fft.fftfreq(frame_size, d=1/fs)
    freqs = freqs[:frame_size // 2]

    for frame in frames:
        windowed_frame = frame * window
        spectrum = np.abs(fft(windowed_frame))
        spectrum = spectrum[:len(spectrum)//2] + 1e-12

        centroid = np.sum(freqs * spectrum) / np.sum(spectrum)
        flatness = np.exp(np.mean(np.log(spectrum))) / np.mean(spectrum)

        cep = np.abs(ifft(np.log(spectrum)))
        cep_mean = np.mean(cep)

        centroids.append(centroid)
        flatnesses.append(flatness)
        cepstra.append(cep_mean)

    return (
        np.mean(centroids),
        np.mean(flatnesses),
        np.mean(cepstra)
    )



Načtení databáze + extrakce příznaků

Diagnózy byly automaticky načítány z hlavičkových souborů databáze VOICED.
Vzhledem k nejednotnému zápisu diagnóz byly normalizovány do několika základních kategorií
(healthy, dysphonia, vocal cord pathology, other pathology).
Tento krok může vést ke sloučení některých méně častých diagnóz do obecné kategorie.

In [8]:
data_dir = "voiced_database/"  

features = []
labels_binary = []
labels_multi = []

example_signal = None
example_fs = None

for file in os.listdir(data_dir):
    if file.endswith(".hea"):
        record_name = file.replace(".hea", "")
        record_path = os.path.join(data_dir, record_name)

        try:
            record = wfdb.rdrecord(record_path)
            signal = record.p_signal[:, 0]
            fs = record.fs

            if example_signal is None:
                example_signal = signal
                example_fs = fs

            # Diagnóza z hlavičky
            diag = "unknown"
            for comment in record.comments:
                if "diagnosis" in comment.lower():
                    diag = comment.split(":")[-1].strip()

            diagnosis = normalize_diagnosis(diag)

            # Příznaky
            centroid, flatness, cepstrum = spectral_features(signal, fs)
            rms, zcr, var = time_domain_features(signal)

            features.append([
                centroid,
                flatness,
                cepstrum,
                rms,
                zcr,
                var
            ])

            labels_binary.append(0 if diagnosis == "healthy" else 1)
            labels_multi.append(diagnosis)

        except Exception as e:
            print(f"Chyba při zpracování {record_name}: {e}")


FileNotFoundError: [WinError 3] Systém nemůže nalézt uvedenou cestu: 'voiced_database/'

Vizualizace časového průběhu

Graf znázorňuje časový průběh hlasového signálu samohlásky „a“
v délce jedné sekundy. Je patrná kvaziperiodická struktura signálu,
typická pro fonaci lidského hlasu.
Tento typ signálu je vhodný pro další analýzu ve frekvenční a kepstrální oblasti.

In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(example_signal[:example_fs])
plt.title("Časový průběh hlasového signálu (1 s)")
plt.xlabel("Vzorky")
plt.ylabel("Amplituda")
plt.grid(True)
plt.show()

Příprava dat

In [ ]:
X = np.array(features)
y_bin = np.array(labels_binary)
y_multi = np.array(labels_multi)

scaler = StandardScaler()
X = scaler.fit_transform(X)

ALL_CLASSES = [
    "healthy",
    "dysphonia",
    "vocal_cord_pathology",
    "other_pathology"
]

BINÁRNÍ KLASIFIKACE


Binární klasifikace byla provedena pomocí náhodného lesa (Random Forest).
Výsledky jsou hodnoceny pomocí matice záměn a metrik precision, recall a F1-score.
Model dosahuje smysluplných výsledků, což potvrzuje, že zvolené příznaky
nesou informaci o patologii hlasu.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y_bin,
    test_size=0.3,
    random_state=42,
    stratify=y_bin
)

clf_bin = RandomForestClassifier(
    n_estimators=100,
    random_state=42
)
clf_bin.fit(X_train, y_train)

y_pred = clf_bin.predict(X_test)

labels_bin = np.unique(y_test)

print("BINÁRNÍ KLASIFIKACE (ZDRAVÝ × PATOLOGICKÝ)")

labels_bin = np.unique(y_test)

if len(labels_bin) > 1:
    print(classification_report(y_test, y_pred, labels=labels_bin))
else:
    print("Pouze jedna třída v datech:", labels_bin[0])

cm_bin = confusion_matrix(y_test, y_pred, labels=labels_bin)
print("Matice záměn:")
print(cm_bin)





VÍCETŘÍDNÍ KLASIFIKACE

Při vícetřídní klasifikaci bylo zjištěno, že všechny analyzované záznamy
byly na základě automatického čtení hlavičkových souborů zařazeny
do kategorie other_pathology.
Tento jev je způsoben nejednotným nebo chybějícím zápisem diagnóz v databázi VOICED.
Výsledná matice záměn proto obsahuje pouze jednu třídu, což odpovídá struktuře dat
a zvolenému způsobu normalizace diagnóz.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y_multi,
    test_size=0.3,
    random_state=42,
    stratify=y_multi
)

clf_multi = RandomForestClassifier(
    n_estimators=150,
    random_state=42
)
clf_multi.fit(X_train, y_train)

y_pred = clf_multi.predict(X_test)

labels_multi = np.unique(y_test)

print("\nVÍCETŘÍDNÍ KLASIFIKACE PORUCH")

labels_multi = np.unique(y_test)

if len(labels_multi) > 1:
    print(classification_report(y_test, y_pred, labels=labels_multi))
else:
    print("Pouze jedna třída v datech:", labels_multi[0])

cm_multi = confusion_matrix(y_test, y_pred, labels=labels_multi)
print("Matice záměn:")
print(cm_multi)



In [ ]:
import seaborn as sns

labels_present = np.unique(y_test)

cm = confusion_matrix(
    y_test,
    y_pred,
    labels=labels_present
)

plt.figure(figsize=(4, 4))
sns.heatmap(
    cm_multi,
    annot=True,
    fmt="d",
    cmap="Blues",
    xticklabels=labels_multi,
    yticklabels=labels_multi
)
plt.show()



In [ ]:
import seaborn as sns

plt.figure(figsize=(7, 5))

# zdravé
idx_healthy = y_bin == 0
plt.scatter(
    X[idx_healthy, 0],   # spektrální centroid
    X[idx_healthy, 3],   # RMS
    color="tab:blue",
    alpha=0.6,
    label="Zdravý"
)

# patologické
idx_path = y_bin == 1
plt.scatter(
    X[idx_path, 0],
    X[idx_path, 3],
    color="tab:red",
    alpha=0.6,
    label="Patologický"
)

plt.xlabel("Spektrální centroid (normalizovaný)")
plt.ylabel("RMS (normalizované)")
plt.title("Distribuce příznaků: zdravý vs patologický hlas")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
